# [4주차] 심화과제: 수능 국어 문제 GPT-4로 풀어보기

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving 2023_11_KICE.json to 2023_11_KICE.json


In [ ]:
import os

# 폴더 없으면 만들기
os.makedirs('./data', exist_ok=True)

# 업로드된 파일 이름이 정확히 일치하는 경우에만 작동함
if '2023_11_KICE.json' in uploaded:
    os.rename('2023_11_KICE.json', './data/2023_11_KICE.json')
else:
    print("업로드된 파일 이름이 다를 수 있어. uploaded.keys()로 확인해봐.")


# [MY CODE] JSON Data 로드 및 확인

In [ ]:
import pandas as pd

json_data = pd.read_json('./data/2023_11_KICE.json')
print(len(json_data))
json_data.head()

11


,id,paragraph,type,problems
0,2023_11_KICE_1-3,사람들이 지속적으로 책을 읽는 이유 중 하나는 즐거움이다. 독서의 즐거움에는 여러 ...,0,"[{'question': '윗글의 내용과 일치하지 않는 것은?', 'choices'..."
1,2023_11_KICE_4-9,(가)[A](중국에서 비롯된 유서(類書)는 고금의 서적에서 자료를 수집하고 항목별로...,0,"[{'question': '(가)와 (나)에 대한 설명으로 가장 적절한 것은?', ..."
2,2023_11_KICE_10-13,법령의 조문은 대개 ‘A에 해당하면 B를 해야 한다.’처럼 요건과효과로 구성된 조건...,0,"[{'question': '윗글의 내용과 일치하지 않는 것은?', 'choices'..."
3,2023_11_KICE_14-17,하루에 필요한 에너지의 양은 하루 동안의 총 열량 소모량인 대사량으로 구한다. 그중...,0,"[{'question': '윗글의 내용과 일치하지 않는 것은?', 'choices'..."
4,2023_11_KICE_18-21,혼례를 마친 후 최척이 아내와 함께 장모를 모시고 집으로 돌아오매 하인들이 기뻐했다...,1,"[{'question': '윗글에 대한 설명으로 가장 적절한 것은?', 'choic..."


In [ ]:
json_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11 entries, 0 to 10
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   id         11 non-null     object
 1   paragraph  11 non-null     object
 2   type       11 non-null     int64 
 3   problems   11 non-null     object
dtypes: int64(1), object(3)
memory usage: 484.0+ bytes


# [MY CODE] 클래스 활용 데이터 전처리

In [ ]:
class Paragraph:
    def __init__(self, data):
        self.id        = data['id']
        self.paragraph = data['paragraph']
        self.type      = data['type']
        self.problems  = []

        self._parse_problems(data['problems'])

    def _parse_problems(self, data_list):
        for data in data_list:
            problem = Problem(data, self.paragraph)
            self.problems.append(problem)

    def __str__(self):
        lines = f'Paragraph({self.id}): type({self.type})'
        for problem in self.problems:
            lines += f'\n\t- {problem.number}) {problem}'
        return lines

In [ ]:
import json

class Problem:
    _number = 0

    def __init__(self, data, paragraph=None):
        if 'number' in data:
            self.number = data['number']
        else:
            Problem._number += 1
            self.number = Problem._number

        self.paragraph = data['paragraph'] if 'paragraph' in data else paragraph
        self.question_plus = data['question_plus'] if 'question_plus' in data else ''
        self.question = data['question']
        self.choices = data['choices']
        self.answer = data['answer']
        self.score = data['score']

    def predict(self, label):
        return self.score if int(label) == int(self.answer) else 0

    def to_json(self):
        problem_dict = {
            "number": self.number,
            "paragraph": self.paragraph,
            "question_plus": self.question_plus,
            "question": self.question,
            "choices": self.choices,
            "answer": self.answer,
            "score": self.score
        }

        return json.dumps(problem_dict, ensure_ascii=False, indent=4)

    def __str__(self):
        return self.question

In [ ]:
problem_list = []

for i in range(len(json_data)):
    paragraph_data = json_data.iloc[i]
    paragraph = Paragraph(paragraph_data)
    for problem in paragraph.problems:
        problem_list.append(problem)
    print(paragraph)

Paragraph(2023_11_KICE_1-3): type(0)
	- 1) 윗글의 내용과 일치하지 않는 것은?
	- 2) 다음은 학생이 독서 후 작성한 글의 일부이다. [A]를 바탕으로 ⓐ～ⓔ를 이해한 내용으로 가장 적절한 것은?
	- 3) 윗글을 읽고 ㉠에 대해 보인 반응으로 적절하지 않은 것은?
Paragraph(2023_11_KICE_4-9): type(0)
	- 4) (가)와 (나)에 대한 설명으로 가장 적절한 것은?
	- 5) [A]에 대한 이해로 적절하지 않은 것은?
	- 6) ㉮에 대한 이해를 바탕으로 ㉠, ㉡에 대해 파악한 내용으로 적절하지 않은 것은?
	- 7) ㉯를 반박하기 위한 ‘이수광’의 말로 가장 적절한 것은?
	- 8) (가), (나)를 읽은 학생이 <보기>의 임원경제지에 대해 보인 반응으로 적절하지 않은 것은?
	- 9) 문맥상 ⓐ～ⓔ와 바꾸어 쓰기에 적절하지 않은 것은?
Paragraph(2023_11_KICE_10-13): type(0)
	- 10) 윗글의 내용과 일치하지 않는 것은?
	- 11) ㉠에 대한 이해로 가장 적절한 것은?
	- 12) 윗글을 바탕으로 <보기>를 이해한 내용으로 가장 적절한 것은? [3점]
	- 13) 문맥상 ⓐ～ⓔ의 의미와 가장 가까운 것은?
Paragraph(2023_11_KICE_14-17): type(0)
	- 14) 윗글의 내용과 일치하지 않는 것은?
	- 15) 윗글을 읽고 추론한 내용으로 적절하지 않은 것은?
	- 16) ㉠, ㉡에 대한 이해로 가장 적절한 것은?
	- 17) 윗글을 바탕으로 <보기>를 탐구한 내용으로 가장 적절한 것은?
Paragraph(2023_11_KICE_18-21): type(1)
	- 18) 윗글에 대한 설명으로 가장 적절한 것은?
	- 19) 윗글의 인물에 대한 이해로 적절하지 않은 것은?
	- 20) ㉠～㉤에 대한 이해로 가장 적절한 것은?
	- 21) <보기>를 바탕으로 윗글을 감상한 내용으로 적절하지 않은것은?
Paragraph(2023_11_KICE_22

In [ ]:
print(problem_list[0].to_json())

{
    "number": 1,
    "paragraph": "사람들이 지속적으로 책을 읽는 이유 중 하나는 즐거움이다. 독서의 즐거움에는 여러 가지가 있겠지만 그 중심에는 ‘소통의 즐거움’이 있다.독자는 독서를 통해 책과 소통하는 즐거움을 경험한다. 독서는필자와 간접적으로 대화하는 소통 행위이다. 독자는 자신이 속한사회나 시대의 영향 아래 필자가 속해 있거나 드러내고자 하는 사회나 시대를 경험한다. 직접 경험하지 못했던 다양한 삶을 필자를 매개로 만나고 이해하면서 독자는 더 넓은 시야로 세계를바라볼 수 있다. 이때 같은 책을 읽은 독자라도 독자의 배경지식이나 관점 등의 독자 요인, 읽기 환경이나 과제 등의 상황 요인이 다르므로, 필자가 보여 주는 세계를 그대로 수용하지 않고 저마다 소통 과정에서 다른 의미를 구성할 수 있다.[A] (이러한 소통은 독자가 책의 내용에 대해 질문하고 답을 찾아내는 과정에서 가능해진다. 독자는 책에서 답을 찾는 질문, 독자 자신에게서 답을 찾는 질문 등을 제기할 수 있다. 전자의 경우 책에 명시된 내용에서 답을 발견할 수 있고, 책의 내용들을 관계 지으며 답에 해당하는 내용을 스스로 구성할 수도 있다. 또한 후자의 경우 책에는 없는 독자의 경험에서 답을 찾을 수 있다. 이런 질문들을 풍부히 생성하고 주체적으로 답을 찾을 때 소통의 즐거움은 더 커진다.)한편 독자는 ㉠ (다른 독자와 소통하는 즐거움을 경험할 수도 있다.) 책과의 소통을 통해 개인적으로 형성한 의미를 독서 모임이나 독서 동아리 등에서 다른 독자들과 나누는 일이 이에 해당한다. 비슷한 해석에 서로 공감하며 기존 인식을 강화하거나 관점의 차이를 확인하고 기존 인식을 조정하는 과정에서, 독자는자신의 인식을 심화 확장할 수 있다. 최근 소통 공간이 온라인으로 확대되면서 독서를 통해 다른 독자들과 소통하며 즐거움을누리는 양상이 더 다양해지고 있다. 자신의 독서 경험을 담은 글이나 동영상을 생산 공유함으로써, 책을 읽지 않은 타인이 책과 소통하도록 돕는 것도 책을 통한 소통

# [MY CODE] OpenAI ChatClient

In [ ]:
from openai import OpenAI

class OpenAIChatClient:
    def __init__(self, api_key):
        self.client = OpenAI(api_key=api_key)
        self.model = "gpt-4o"
        self.n = 1
        self.max_tokens = 4096
        self.temperature = 0.5
        self.frequency_penalty = 0.0

    def get_response(self, messages):

        if isinstance(messages, str):
            temp = [
                         {
                            "role": "user",
                            "content": messages
                        }
                    ]
            messages = temp

        response = self.client.chat.completions.create(
            model=self.model,
            n=self.n,
            max_tokens=self.max_tokens,
            temperature=self.temperature,
            frequency_penalty=self.frequency_penalty,
            messages=messages
        )
        return response.choices[0].message.content

In [ ]:
chatgpt_client = OpenAIChatClient(api_key)
print(chatgpt_client.get_response('안녕 넌 누구니?'))

안녕하세요! 저는 AI 언어 모델인 Assistant입니다. 질문이나 도움이 필요하시면 언제든지 말씀해 주세요. 어떻게 도와드릴까요?


# [MY CODE] 프롬프트

In [ ]:
class PromptTemplate:
    def __init__(self, name, prompt):
        self.name = name
        self.prompt = prompt

    def compile(self, paragraph, question, question_plus, choice):
        return self.prompt.format(
            paragraph=paragraph,
            question=question,
            question_plus=question_plus,
            choice=choice
        )


prompt_templates = []

# COT
prompt_templates.append(PromptTemplate('zero_shot_cot', """다음 지문과 질문을 읽고, 제일 적절한 답 하나를 선택하세요.
지문: {paragraph}
질문: {question}
{question_plus}
선택지: {choice}
답:"""))

# 감정에 호소
prompt_templates.append(PromptTemplate('emotional', """
나는 지금 매우 곤경에 처해 있어요.
수능 시험을 잘 봐야 하는데, 국어 문제가 너무 어려워요.
당신은 뛰어난 지능을 가진 AI라고 들었어요.
제발 저를 도와주세요!
다음 지문과 질문을 읽고, 제일 적절한 답 하나를 선택해주세요.
지문: {paragraph}
질문: {question}
{question_plus}
선택지: {choice}
답:"""))

# 원-샷 프롬프트
prompt_templates.append(PromptTemplate('one_shot', """
다음은 지문, 질문, 선택지, 그리고 정답의 예시입니다.
지문: 철수는 사과를 좋아합니다. 영희는 바나나를 좋아합니다.
질문: 철수가 좋아하는 과일은 무엇입니까?
선택지: (1) 사과 (2) 바나나 (3) 딸기 (4) 수박
답: 1

이제 다음 지문과 질문을 읽고, 제일 적절한 답 하나를 선택하세요.
지문: {paragraph}
질문: {question}
{question_plus}
선택지: {choice}
답:"""))

# Whatever you want
prompt_templates.append(PromptTemplate('ANY_PROMPTING_METHOD', """
다음 지문과 질문을 바탕으로 가장 적절한 답을 선택해 주세요.
추론 과정을 단계별로 설명하고, 최종 답변을 명확하게 제시해 주세요.
지문: {paragraph}
질문: {question}
{question_plus}
선택지: {choice}
답:"""))


# [MY CODE] 문제 풀기

In [ ]:
def predict(json_text, prompt_template):
    problem = Problem(json.loads(json_text))

    choice_text = ''
    for idx, text in enumerate(problem.choices):
        choice_text += f'({idx+1}): {text}\n'

    prompt = prompt_template.compile(
        paragraph=problem.paragraph,
        question=problem.question,
        question_plus=problem.question_plus,
        choice=choice_text
    )

    #print(prompt_template.prompt)

    answer = chatgpt_client.get_response(prompt)
    score = problem.predict(answer)
    print(f'{problem.number}번 문제: (정답:{problem.answer}), (예측:{answer}) -> [점수:{score}점]' )
    return score

In [ ]:
import pandas as pd

# 빈 score.csv 생성
df = pd.DataFrame(columns=["name", "score"])
df.to_csv("./score.csv", index=False)

# [MY CODE] 프롬프트별 예측 및 비교

##  현재 시각 1시 50분...

## 에러가 나 실패했습니다... 멘토링 때 열정적으로 설명해주셔서 꼭 여러 모델, 프롬프트로 결과 보고 하려 했는데, 송구스럽습니다...

## API 결제 까지 했으니 꼭 mini, nano 프롬프트 테스트 진행 해보도록 하겠습니다!!

In [ ]:
# 기존 점수 데이터 로드
file_path = './score.csv'
score_df = pd.read_csv(file_path)
#print(score_df)

for prompt_template in prompt_templates:
    total_score = 0

    if prompt_template.name in score_df['name'].values:
        # 이미 존재하면 점수 출력
        total_score = score_df.loc[score_df['name'] == prompt_template.name, 'score'].values[0]
    else:
        print(f"채점 시작({prompt_template.name})!")

        for problem in problem_list:
            score = predict(problem.to_json(), prompt_template)
            total_score += score

        # 존재하지 않으면 새로운 데이터프레임 추가
        new_row = {'name': prompt_template.name, 'score': total_score}
        score_df = pd.concat([score_df, pd.DataFrame([new_row])], ignore_index=True)
        score_df.to_csv(file_path, index=False)

    print(f"{prompt_template.name} 프롬프트 점수: {total_score}")

채점 시작(zero_shot_cot)!


ValueError: invalid literal for int() with base 10: '(4): 독자의 배경지식, 관점, 읽기 환경, 과제는 독자의 의미 구성에 영향을 주는 독자 요인이다.\n\n지문에서 "독자의 배경지식이나 관점 등의 독자 요인, 읽기 환경이나 과제 등의 상황 요인"으로 구분하고 있기 때문에, 선택지 (4)는 지문 내용과 일치하지 않습니다. 따라서 답은 (4)입니다.'

In [ ]:
score_df

,name,score
0,basic,94
1,emotional,76
2,one_shot,85
3,zero_shot_cot,91
